
# Aliasing Diagnostics for Fourier KdV Solver

Runs a handful of coarse KdV simulations (aliased vs dealiased), samples the
FFT spectra at fixed times, and stores the results as tidy Parquet tables for
plotting.


In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

from spectral.tdp import KdVSolver, get_time_integrator, two_soliton_initial

# --------------------------------------------------------------------------- #
# Configuration
# --------------------------------------------------------------------------- #

DATA_DIR = Path("data/A2/ex_e")
DATA_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_SPECTRA = DATA_DIR / "spectra.parquet"

L = 20.0  # half domain length (x ∈ [-L, L))
T_FINAL = 12.0
SAVE_EVERY = 40  # store every N solver steps
SAFETY = 0.35  # CFL safety factor
INTEGRATOR_NAME = "rk4"

# Resolution / dealias combinations to explore
SCENARIOS = [
    ("N64_alias", 64, False),
    ("N64_dealias", 64, True),
    ("N96_alias", 96, False),
    ("N96_dealias", 96, True),
    ("N128_alias", 128, False),
    ("N128_dealias", 128, True),
]

# Two-soliton collision seed: produces spectral activity near the Nyquist limit
C1, X01 = 1.0, -6.0
C2, X02 = 0.4, 6.0


# --------------------------------------------------------------------------- #
# Helper routines
# --------------------------------------------------------------------------- #


def stable_dt(N: int, L: float, u_max: float, *, dealias: bool) -> float:
    """Return a stable timestep (with safety margin) for each configuration."""
    dt_est = KdVSolver.stable_dt(
        N, L, u_max, integrator_name=INTEGRATOR_NAME, dealiased=dealias
    )
    if not np.isfinite(dt_est):
        return 1e-3
    return SAFETY * float(dt_est)


# --------------------------------------------------------------------------- #
# Main loop
# --------------------------------------------------------------------------- #

spectra_frames: list[pd.DataFrame] = []

for label, N, use_dealias in SCENARIOS:
    print(f"Running {label:>12s} | N={N:3d}, dealias={use_dealias}")

    solver = KdVSolver(N, L, dealias=use_dealias)
    x = solver.x
    u0 = two_soliton_initial(x, C1, X01, C2, X02)
    dt = stable_dt(N, L, float(np.max(np.abs(u0))), dealias=use_dealias)
    integrator = get_time_integrator(INTEGRATOR_NAME)

    t_saved, u_saved = solver.solve(
        u0.copy(),
        T_FINAL,
        dt,
        save_every=SAVE_EVERY,
        integrator=integrator,
    )

    idx = np.arange(N // 2 + 1, dtype=int)
    k_vals = pd.Series(solver.k[idx], index=idx)

    fft_vals = np.fft.fft(u_saved, axis=1)[:, idx]
    magnitudes = np.abs(fft_vals)

    df = pd.DataFrame(magnitudes, columns=idx)
    df["t"] = t_saved
    tidy = df.melt(id_vars="t", var_name="mode_index", value_name="abs_u_hat").assign(
        scenario=label,
        N=N,
        dealias=use_dealias,
        dt=dt,
        L=L,
    )
    tidy["mode_index"] = tidy["mode_index"].astype(int)
    tidy["k"] = tidy["mode_index"].map(k_vals)
    tidy["k_abs"] = tidy["k"].abs()

    spectra_frames.append(tidy)

spectra_all = pd.concat(spectra_frames, ignore_index=True)

spectra_all.to_parquet(OUTPUT_SPECTRA, index=False)

print(f"\nSaved spectra  → {OUTPUT_SPECTRA}")